In [1]:
import pandas as pd
import os
import datetime
import dateutil
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

# df stores historic stock price data

In [4]:
files = os.listdir('.')
datafiles = [f for f in files if 'csv' in f]
datafiles.reverse()

df = pd.DataFrame()
for dataf in datafiles:
    df = pd.concat([df, pd.read_csv('./' + dataf)])

df = df.reset_index()
df['Date2'] = df['Date'].apply(lambda x:dateutil.parser.parse(x))

In [6]:
df.head()

,index,Date,Open,High,Low,Close,Volume,Adj Close,Date2
0,0,2017-02-01,132.250000,133.490005,130.679993,133.229996,42482700,133.229996,2017-02-01
1,1,2017-01-31,130.169998,130.660004,129.520004,130.320007,19444000,130.320007,2017-01-31
2,2,2017-01-30,131.580002,131.580002,129.600006,130.979996,18891400,130.979996,2017-01-30
3,3,2017-01-27,132.679993,132.949997,131.080002,132.179993,19493900,132.179993,2017-01-27
4,4,2017-01-26,131.630005,133.139999,131.440002,132.779999,19818100,132.779999,2017-01-26


# a: announcement date

In [7]:
dates = pd.read_excel('FB quarterly report dates.xlsx')

In [8]:
dates.head()

,sec,Unnamed: 1,announcement
0,NaT,NaN,2017-02-01
1,2016-11-03,5.0,2016-11-02
2,2016-07-28,5.0,2016-07-27
3,2016-04-28,5.0,2016-04-27
4,2016-01-28,5.0,2016-01-27


In [11]:
a = dates['announcement']

# define cal_an_close to extract price information for n days before or after the announcement date

In [9]:
def add_n_day(x, n):
    xn = x + datetime.timedelta(days=n)
    if xn.weekday() == 5:
        return xn + datetime.timedelta(days=2)
    elif xn.weekday() == 6:
        return xn + datetime.timedelta(days=1)
    else:
        return xn
    

def cal_an_close(a, n):
    
    an = a.apply(add_n_day, n=n)
    an_close = []

    for test_date in an:
        try:
            an_close.append(df[df['Date2'] == test_date]['Close'].tolist()[0])
        except:
            an_close.append(0)
    return an_close

# calculate the price of day before(-1), day(0), and day after(1,2,...)

In [23]:
df_price = pd.DataFrame([a.tolist(), cal_an_close(a,-1), cal_an_close(a,0), cal_an_close(a,1)]).transpose()

df_price.columns = ['date','daybefore','day','dayafter']

df_price2 = df_price[df_price['daybefore']<df_price['day']]

df_price3 = df_price2[df_price2['day']<df_price2['dayafter']]

print 'day before < day: ', len(df_price2)

print 'day before < day < n days after: ', len(df_price3)

price_difference = df_price3['dayafter'] - df_price3['day']

#print price_difference

print 'difference mean', np.array(price_difference).mean()

print 'difference std', np.array(price_difference).std()

day before < day:  11
day before < day < n days after:  8
difference mean 4.105001375
difference std 2.42679359521
